## Challenge Data Engineer
##### Authored by: Carlos Granados
##### Creation date:     29th March of 2022

### Carga de Datos

In [130]:
import urllib.parse
import urllib.request
import json
import pandas as pd

PROJECT = "houm"
# API_KEY="578AS74EQP9JY69RRFFPFB7QY"
API_KEY = "5VK6Q85UJWGV7ZGZ5V2GSTEF8"

sources = {
    "properties": "./files/properties.csv",
    "users": "./files/users.csv",
    "visits": "./files/visits.csv"
}
SEP = ","


def get_file_to_df():
    properties = pd.read_csv(sources.get("properties"), sep=SEP)
    users = pd.read_csv(sources.get("users"), sep=SEP)
    visits = pd.read_csv(sources.get("visits"), sep=SEP)
    return properties, users, visits


# metodo para extraer consumir la API
def getWeatherForecast(lat, lng, start_date, end_date):
    requestUrl = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lng}/{start_date}/{end_date}?key={API_KEY}&contentType=json&include=days"
    print(requestUrl)
    # print('Weather requestUrl={requestUrl}'.format(requestUrl=requestUrl))
    try:
        req = urllib.request.urlopen(requestUrl)
    except:
        # print("Could not read from:"+requestUrl);
        return []
    rawForecastData = req.read()
    req.close()
    return json.loads(rawForecastData)


# metodo para extraer consumir la API
def getWeatherForecast(lat, lng, start_date, end_date):
    requestUrl = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lng}/{start_date}/{end_date}?key={API_KEY}&contentType=json&include=days"
    print(requestUrl)
    # print('Weather requestUrl={requestUrl}'.format(requestUrl=requestUrl))
    try:
        req = urllib.request.urlopen(requestUrl)
    except:
        # print("Could not read from:"+requestUrl);
        return []
    rawForecastData = req.read()
    req.close()
    return json.loads(rawForecastData)


# Metodo para agrupar la información extraida de la API y a su vez agregarla a la data entregada.
def get_info_weather(df):
    weather = pd.DataFrame()
    for index, row in df.iterrows():
        weatherdata = getWeatherForecast(row["latitude"], row["longitude"], row["begin_date_str"], row["end_date_str"])
        days = weatherdata["days"]
        latitude = weatherdata["latitude"]
        longitude = weatherdata["longitude"]
        # print(days)
        for day in days:
            # print("latitude=" + str(latitude))
            try:
                conditions = day["icon"]

            except:
                conditions = "N/a"

            frame_weather = pd.DataFrame([[row["user_id"],
                                           row["localidad"],
                                           row["status"],
                                           latitude,
                                           longitude,
                                           day['datetime'],
                                           conditions,
                                           day["temp"],
                                           row["begin_date_str"],
                                           row["end_date_str"]
                                           ]], columns=['user_id', 'localidad', 'status', 'latitude',
                                                        'longitude', 'datetime', 'conditions', 'temp',
                                                        'begin_date_str', 'end_date_str'
                                                        ])

        frame_weather = frame_weather.groupby(["user_id", "localidad", "status",
                                               "latitude", "longitude", "datetime",
                                               "conditions"]).mean().reset_index()
        weather = weather.append(frame_weather, ignore_index=True)
    return weather

In [7]:
properties, users, visits = get_file_to_df()

#### Extract number of visits

In [8]:
visits.head()

,scheduled_id,property_id,begin_date,end_date,type_visit,status
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled


In [9]:
visits = visits.drop_duplicates(subset="scheduled_id").reset_index(drop=True)

In [10]:
grouped_visits = visits.groupby("status").count()

#### ¿Cuántas visitas se realizaron en total?
se puede observar que hay 286 visitas canceladas y 139 visitas concretadas

In [12]:
grouped_visits[["scheduled_id"]]

,scheduled_id
status,
Cancelled,286
Done,139


##### ¿Cuál es el promedio de propiedades por usuario?

In [13]:
properties.head()

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.058040,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,2,departamento,Rental & Sale,1,1,2,NaN,1,False,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
2,3,departamento,Rental,2,2,2,NaN,0,False,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia
3,4,departamento,Rental & Sale,3,2,1,0.0,0,False,4.721716,-74.057976,Suba,Bogotá,Región De Cundinamarca,Colombia
4,5,departamento,Rental & Sale,2,3,2,0.0,1,False,4.688163,-74.043300,Usaquen,Bogotá,Región De Cundinamarca,Colombia


In [14]:
users.head()

,property_id,user_id,first_name,last_name,address
0,31,1,Josephine,Darakjy,4 B Blue Ridge Blvd
1,34,2,Art,Venere,8 W Cerritos Ave #54
2,48,3,Lenna,Paprocki,639 Main St
3,27,4,Donette,Foller,34 Center St
4,40,5,Simona,Morasca,3 Mcauley Dr


In [15]:
join_users_prop = users.merge(properties, on='property_id', how='left')

In [42]:
join_users_prop.head()

,property_id,user_id,first_name,last_name,address,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,31,1,Josephine,Darakjy,4 B Blue Ridge Blvd,departamento,Rental & Sale,2,2,2,NaN,1,False,4.694191,-74.043625,Usaquen,Bogotá,Región De Cundinamarca,Colombia
1,34,2,Art,Venere,8 W Cerritos Ave #54,departamento,Rental,1,1,0,NaN,0,False,4.618363,-74.073500,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
2,48,3,Lenna,Paprocki,639 Main St,departamento,Rental,3,1,0,NaN,1,False,4.755693,-74.088680,Suba,Bogotá,Región De Cundinamarca,Colombia
3,27,4,Donette,Foller,34 Center St,departamento,Rental,2,2,0,NaN,0,False,4.674879,-74.134390,Fontibon,Bogotá,Región De Cundinamarca,Colombia
4,40,5,Simona,Morasca,3 Mcauley Dr,departamento,Rental,2,2,2,NaN,0,False,4.644231,-74.107834,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia


In [44]:
grouped_data = join_users_prop.groupby("user_id").count()

##### La data nos indica que en promedio cada usuario tiene una propiedad.

In [17]:
grouped_data[["property_id"]].mean()

property_id    1.0
dtype: float64

#### ¿Cuál era la temperatura promedio de todas las visitas que realizó el usuario con ID 2?

In [18]:
grouped_data.reset_index()

,user_id,property_id,first_name,last_name,address,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
1,2,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
2,3,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
3,4,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
4,5,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
76,77,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
77,78,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
78,79,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1


##### Join Data

In [19]:
visits_joined = users.merge(visits, on='property_id', how='left')

In [20]:
all_joined = visits_joined.merge(properties, on='property_id', how='left')

#### agregar strings para fechas

In [21]:
all_joined['begin_date_str'] = all_joined['begin_date'].str[:10]
all_joined['end_date_str'] = all_joined['end_date'].str[:10]

In [76]:
weather = get_info_weather(all_joined)

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/4.6941915%2C-74.043625/2022-01-15/2022-01-15?key=5VK6Q85UJWGV7ZGZ5V2GSTEF8&contentType=json&include=days
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/4.6941915%2C-74.043625/2022-01-17/2022-01-17?key=5VK6Q85UJWGV7ZGZ5V2GSTEF8&contentType=json&include=days
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/4.6941915%2C-74.043625/2022-01-16/2022-01-16?key=5VK6Q85UJWGV7ZGZ5V2GSTEF8&contentType=json&include=days
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/4.6941915%2C-74.043625/2022-01-14/2022-01-14?key=5VK6Q85UJWGV7ZGZ5V2GSTEF8&contentType=json&include=days
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/4.6183634%2C-74.0735/2022-01-29/2022-01-29?key=5VK6Q85UJWGV7ZGZ5V2GSTEF8&contentType=json&include=days
https://weather.visualcrossing.com/VisualCrossingWebServi

In [67]:
weather_filtered = weather[weather.user_id == 2] 

##### Se puede observar que para el usuario de id = 2 la temperatura promedio de todas las visitas es 57.8428 F

In [70]:
weather_filtered["temp"].mean()

57.84285714285714

In [ ]:
weather

,user_id,localidad,status,latitude,longitude,datetime,conditions,temp
0,1,Usaquen,Cancelled,4.694191,-74.043625,2022-01-15,rain,55.5
1,1,Usaquen,Cancelled,4.694191,-74.043625,2022-01-17,rain,56.1
2,1,Usaquen,Cancelled,4.694191,-74.043625,2022-01-16,rain,55.7
3,1,Usaquen,Done,4.694191,-74.043625,2022-01-14,rain,56.0
4,2,Teusaquillo,Cancelled,4.618363,-74.073500,2022-01-29,rain,56.1
...,...,...,...,...,...,...,...,...
420,80,Rafael Uribe Uribe,Done,4.587935,-74.127815,2022-02-27,rain,57.2
421,80,Rafael Uribe Uribe,Cancelled,4.587935,-74.127815,2022-02-28,rain,56.2
422,80,Rafael Uribe Uribe,Done,4.587935,-74.127815,2022-02-26,rain,58.3
423,80,Rafael Uribe Uribe,Cancelled,4.587935,-74.127815,2022-02-27,rain,57.2


#### ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?
##### Se puede observar que la temperatura promedio en tiempo en los dias con lluvia es: 57.302902374670175
###### Sin embargo esta no es una medida acertada, dado que por cada día pueden existir varias mediciones de temperatura, dado que se estan validando las diferentes zonas y regiones lo cual hace que no se pueda dar con certeza el valor promedio, sin embargo estadisticamente no deberia estar muy distante. 

In [84]:
weather[weather.conditions == "rain"]["temp"].mean()

57.302902374670175

In [129]:
all_joined.groupby("city")["city"].count()

city
Bogotá          342
Cundinamarca     83
Name: city, dtype: int64

##### ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?
###### La temperatura promedio para las VISITAS realizadas en la localidad de suba es: 57.61395348837206

In [85]:
weather[weather.localidad == "Suba"]["temp"].mean()

57.61395348837206